# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver # permite criar o navegador
from selenium.webdriver.common.keys import Keys # permite escrever no navegador
from selenium.webdriver.common.by import By # permite você selecionar itens no navegador

### cotação do Dólar - Euro - Ouro

In [4]:
# abrir o navegador
navegador = webdriver.Chrome()

# entrar no google
navegador.get('https://www.google.com/')

# clicando em aceitar os cookies no inicio
navegador.find_element('xpath','/html/body/div[2]/div[2]/div[3]/span/div/div/div/div[3]/div[1]/button[2]/div').click()

# pesquisar no goolge por "cotação dólar"
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar para real brasileiro')

navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar cotação que tá no google
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# ----------------------------------------------- ## -----------------------------------------

# abrir o navegador
navegador = webdriver.Chrome()

# entrar no google
navegador.get('https://www.google.com/')

# clicando em aceitar os cookies no inicio
navegador.find_element('xpath','/html/body/div[2]/div[2]/div[3]/span/div/div/div/div[3]/div[1]/button[2]/div').click()

# pesquisar no goolge por "cotação euro"
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro para real brasileiro')

navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar cotação que tá no google
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# ----------------------------------------------- ## -----------------------------------------

# abrir o navegador
navegador = webdriver.Chrome()

# entrar no google
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath', '/html/body/div[5]/div[1]/div/div/input[2]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')

navegador.quit()
# ----------------------------------------------- ## -----------------------------------------

In [5]:
cotacao_dolar = round(float(cotacao_dolar),2)
cotacao_euro = round(float(cotacao_euro),2)
cotacao_ouro = round(float(cotacao_ouro),2)

print(f'Cotação atual do dolar R$ {cotacao_dolar:.2f}')
print(f'Cotação atual do euro R$ {cotacao_euro:.2f}')
print(f'Cotação atual do ouro R$ {cotacao_ouro:.2f}')

Cotação atual do dolar R$ 5.20
Cotação atual do euro R$ 5.12
Cotação atual do ouro R$ 284.63


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [9]:
import pandas as pd
df = pd.read_excel('Produtos.xlsx')
df

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [7]:
# atualizar a coluna cotação
df.loc[df['Moeda'] == 'Dólar', 'Cotação'] = cotacao_dolar
df.loc[df['Moeda'] == 'Euro', 'Cotação'] = cotacao_euro
df.loc[df['Moeda'] == 'Ouro', 'Cotação'] = cotacao_ouro

# atualizar preço de compra = preço original * cotação
df['Preço de Compra'] = df['Preço Original'] * df['Cotação']
df['Preço de Compra'] = round(df['Preço de Compra'], 2)

# atualizar preço de venda = preço de compra * margem 
df['Preço de Venda'] = df['Preço de Compra'] * df['Margem']
df['Preço de Venda'] = round(df['Preço de Venda'], 2)
df.head()

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.20,5199.95,1.4,7279.93
1,Carro Renault,4500.00,Euro,5.12,23040.00,2.0,46080.00
2,Notebook Dell,899.99,Dólar,5.20,4679.95,1.7,7955.91
3,IPhone,799.00,Dólar,5.20,4154.80,1.7,7063.16
4,Carro Fiat,3000.00,Euro,5.12,15360.00,1.9,29184.00


### Agora vamos exportar a nova base de preços atualizada

In [8]:
df.to_excel('produtos_atualizado.xlsx', index=False)